# Advent of Code 2023

https://adventofcode.com/2023

In [1]:
# A few standard library imports
import itertools, math, collections, os, functools, json, time, re, bisect
# nice printing
from IPython.display import display, clear_output, HTML

In [2]:
def getData(day, year=2023):
    if not os.path.exists("data-%d" %year):
        os.mkdir("data-%d" %year)
    if os.path.exists("data-%d/day%dinput.txt" %(year, day)):
        with open("data-%d/day%dinput.txt" %(year, day), 'r') as f:
            return f.read()
    import browsercookie, urllib.request
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(browsercookie.firefox()))
    assert b'watercrossing' in opener.open("https://adventofcode.com/%d/" %year).read()
    data = opener.open("https://adventofcode.com/%d/day/%d/input" %(year, day)).read().decode('utf-8')
    with open("data-%d/day%dinput.txt" %(year, day), 'w') as f:
        f.write(data)
    return data

In [3]:
def isNum(s: str) -> bool:
    return all(47 < ord(c) < 58 for c in s)

## Day 1

In [4]:
day1data = getData(1).splitlines()

In [5]:
sum(int(next(c for c in x if isNum(c)) + next(c for c in x[::-1] if isNum(c))) for x in day1data)

54927

In [6]:
z = list(zip(["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"] + [str(x) for x in range(0, 10)], 
              [str(x) for x in range(1, 10)] + [str(x) for x in range(0, 10)]))

In [7]:
sum(int(min([(x.find(c), d) for c, d in z if c in x], key=lambda a: a[0])[1] + 
        min([(x[::-1].find(c[::-1]), d) for c, d in z if c in x], key=lambda a: a[0])[1]) for x in day1data)

54581

## Day 2

In [8]:
day2data = [(int(r.split(":")[0].split(" ")[1]), [collections.Counter(dict((y.split(" ")[1], int(y.split(" ")[0])) for y in x.split(", ")))
                                                  for x in r.split(": ")[1].split("; ")]) for r in getData(2).splitlines()]

In [9]:
sum(c for c, x in day2data if all(max(y[a] for y in x) < b for a, b in (('red', 13), ('green', 14), ('blue', 15))))

2239

In [10]:
sum(math.prod(max(c[a] for c in x) for a in ('red', 'green', 'blue')) for _, x in day2data)

83435

## Day 3

In [11]:
day3data = getData(3).splitlines()
il, jl= len(day3data), len(day3data[0])

In [12]:
sum(int(num) for i in range(il) for j in range(jl) if (j==0 or not isNum(day3data[i][j-1])) and isNum(day3data[i][j])
    and (num := next(day3data[i][j:k] for k in range(j+1,jl+1) if isNum(day3data[i][j:k]) and (k == jl or not isNum(day3data[i][k]))))
    and any(not isNum(day3data[a][b]) and day3data[a][b] != '.' for a in range(max(0,i-1), min(il,i+2)) 
            for b in range(max(0, j-1), min(jl, j+1+len(num)))))

538046

In [13]:
sum(math.prod(gears) for i in range(il) for j in range(jl) if day3data[i][j] == "*"
    and len(gears := [next(int(day3data[k][lx:ly]) for lx in range(l, max(-1, l-6), -1) for ly in range(l+1, min(jl+1, l+6))
                           if isNum(day3data[k][lx:ly]) and (lx == 0 or not isNum(day3data[k][lx-1])) and (ly == jl or not isNum(day3data[k][ly]))) 
                      for k, l in [(ix, j-1) for ix in range(max(0, i-1), min(il, i+2)) if j > 0] + 
                                  [(ix, jx) for ix in range(max(0, i-1), min(il, i+2)) for jx in range(j, min(jl, j+2))
                                   if jx == 0 or not isNum(day3data[ix][jx-1])] if isNum(day3data[k][l])]) == 2)

81709807

## Day 4

In [14]:
day4data = [tuple(set(y for y in x.split(": ")[1].split(" | ")[i].split(" ") if y) for i in range(2)) for x in getData(4).splitlines()]

In [15]:
sum(2**(len(c&w)-1) if len(c&w) else 0 for w, c in day4data)

24160

In [16]:
cards = [[1, len(c&w)] for c, w in day4data]
for i, (c, w) in enumerate(cards):
    for j in range(w):
        cards[i+j+1][0] += c
sum(x[0] for x in cards)

5659035

## Day 5

In [17]:
day5data = [[[int(y) for y in l.split()] for l in x.split(":")[1].strip().splitlines()] for x in getData(5).split("\n\n")]
day5data[0] = day5data[0][0]

In [18]:
min(functools.reduce(lambda cs, m: [next(itertools.chain(d+c-s for d, s, l in m if s <= c < s+l), c) for c in cs], day5data))

324724204

In [19]:
day5data[0] = [[range(day5data[0][2*i], day5data[0][2*i]+day5data[0][2*i+1])] for i in range(len(day5data[0])//2)]

In [20]:
def day5map(css, m):
    coveredRanges = [[range(max(s, c[0]), min(s+l, c[-1]+1)) for c in cs for d, s, l in m if min(s+l, c[-1]+1) > max(s, c[0])] for cs in css]
    destRanges = [[range(max(s, c[0])-s+d, min(s+l, c[-1]+1)-s+d) for c in cs for d, s, l in m if min(s+l, c[-1]+1) > max(s, c[0])] for cs in css]
    notMapped = []
    for tc, cr in zip(css, coveredRanges):
        for d in cr:
            tc = [x for c in tc for x in [range(c[0], min(c[-1]+1, d[0])), range(max(d[-1]+1, c[0]), c[-1]+1)] if x.stop > x.start]
        notMapped.append(tc)
    return [x+y for x, y in zip(destRanges, notMapped)]

In [21]:
min(x[0] for y in functools.reduce(day5map, day5data) for x in y)

104070862

## Day 6

In [22]:
day6data = list(zip(*[[int(x.strip()) for x in r.split(":")[1].split(" ") if x] for r in getData(6).splitlines()]))

In [23]:
math.prod(sum(1 for i in range(t) if i*t-i*i > r) for t, r in day6data)

771628

In [24]:
day6dataPart2 = [int(r.split(":")[1].strip().replace(" ","")) for r in getData(6).splitlines()]

In [25]:
class Day6:
    def __getitem__(self, key):
        return key*day6dataPart2[0] - key*key
    def __len__(self):
        return day6dataPart2[0]//2

In [26]:
(day6dataPart2[0]//2 - bisect.bisect(Day6(), day6dataPart2[1]))*2 + 1

27363861

## Day 7

In [27]:
day7data = [x.split(" ") for x in getData(7).splitlines()]

In [28]:
def day7replace(s):
    s = s.replace("T", "a").replace("J", "b").replace("Q", "c").replace("K", "d").replace("A", "e")
    c = collections.Counter(s).values()
    if 5 in c:
        s = "6" + s
    elif 4 in c:
        s = "5" + s
    elif 3 in c and 2 in c:
        s = "4" + s
    elif 3 in c:
        s = "3" + s
    elif collections.Counter(c)[2] == 2:
        s = "2" + s
    elif 2 in c:
        s = "1" + s
    return int(s, base=16)

In [29]:
day7sorted = sorted(day7data, key=lambda x: day7replace(x[0]))

In [30]:
sum((i+1)*int(bid) for i, (h, bid) in enumerate(day7sorted))

249726565

In [31]:
def day7replacePart2(s):
    s = s.replace("T", "a").replace("J", "1").replace("Q", "c").replace("K", "d").replace("A", "e")
    js = collections.Counter(s)['1'] 
    c = collections.Counter(s.replace("1", "")).values()
    if js == 5 or 5 in c or max(c)+js == 5:
        s = "6" + s
    elif 4 in c or max(c) + js == 4:
        s = "5" + s
    elif 3 in c and 2 in c or (collections.Counter(c)[2] == 2 and js == 1):
        s = "4" + s
    elif 3 in c or (2 in c and js == 1) or js == 2:
        s = "3" + s
    elif collections.Counter(c)[2] == 2:
        s = "2" + s
    elif 2 in c or js == 1:
        s = "1" + s
    return int(s, base=16)

In [32]:
day7sortedPart2 = sorted(day7data, key=lambda x: day7replacePart2(x[0]))

In [33]:
sum((i+1)*int(bid) for i, (h, bid) in enumerate(day7sortedPart2))

251135960

## Day 8

In [34]:
day8data = (getData(8).split("\n\n")[0], 
            dict((x.split(" = ")[0], x.split(" = ")[1].strip("()").split(", ")) for x in getData(8).split("\n\n")[1].split("\n") if x))

In [35]:
c, cn = 0, "AAA"
while cn != "ZZZ":
    cn = day8data[1][cn][0 if day8data[0][c % len(day8data[0])] == 'L' else 1]
    c += 1
c

22411

In [36]:
cns = [x for x in day8data[1].keys() if x.endswith("A")]
orders = [[0] for _ in range(len(cns))]
for i, cn in enumerate(cns):
    c = 0
    while len(orders[i]) != 4:
        cn = day8data[1][cn][0 if day8data[0][c % len(day8data[0])] == 'L' else 1]
        c += 1
        if cn.endswith("Z"):
            orders[i].append(c)

In [37]:
[[x[i+1]-x[i] for i in range(len(x)-1)] for x in orders]

[[16271, 16271, 16271],
 [24253, 24253, 24253],
 [13201, 13201, 13201],
 [14429, 14429, 14429],
 [18113, 18113, 18113],
 [22411, 22411, 22411]]

In [38]:
# they are all the same. I didn't expect that. But that makes it easy.

In [39]:
math.lcm(*(x[1] for x in orders))

11188774513823

## Day 9

In [40]:
day9data = [[int(x) for x in y.split(" ")] for y in getData(9).splitlines()]

In [41]:
nextVals = []
for x in day9data:
    diffs = [x[-1]]
    while not all(y == 0 for y in x):
        x = [x[i+1] - x[i] for i in range(len(x)-1)]
        diffs.append(x[-1])
    nextVals.append(sum(diffs))
sum(nextVals)

1987402313

In [42]:
nextVals = []
for x in day9data:
    diffs = [x[0]]
    while not all(y == 0 for y in x):
        x = [x[i+1] - x[i] for i in range(len(x)-1)]
        diffs.append(x[0])
    nextVals.append(sum((-1)**i*x for i, x in enumerate(diffs)))
sum(nextVals)

900

## Day 10

In [43]:
day10data = getData(10).splitlines()
tileDirs = {"-": ((0, 1), (0, -1)), "|": ((1, 0), (-1, 0)), "J": ((-1, 0), (0, -1)), 
            "7": ((0, -1), (1, 0)), "L": ((-1, 0), (0, 1)), "F": ((1, 0), (0, 1))}
nbs = [(0, 1), (0, -1), (1, 0), (-1, 0)]

In [44]:
start = next((i, j) for i in range(len(day10data)) for j in range(len(day10data[0])) if day10data[i][j] == "S")
connectIntoStart = [(i, j) for i, j, shapes in [(start[0]+1, start[1], "|JL"), (start[0]-1, start[1], "|7F"), 
                                                (start[0], start[1]-1, "-FL"), (start[0], start[1]+1, "-J7")] if day10data[i][j] in shapes]

In [45]:
loop = [start, connectIntoStart[0]]
while loop[-1] != start:
    loop.append(next(nt for i, j in tileDirs[day10data[loop[-1][0]][loop[-1][1]]] if (nt := (loop[-1][0] + i, loop[-1][1] + j)) != loop[-2]))
(len(loop) - 1 ) // 2

6697

In [46]:
loopSet = set(loop)
day10dataPart2 = [[[day10data[i][j], set()] if (i, j) in loopSet else [".", set()] for j in range(len(day10data[0]))] for i in range(len(day10data))]
for d in itertools.cycle((range(len(day10data))[::x],range(len(day10data[1]))[::y])  for x, y in [(1, 1), (1, -1), (-1, 1), (-1, -1)]):
    anyChanges = False
    for i, j in ((x, y) for x in d[0] for y in d[1]):
        if day10dataPart2[i][j][0] == ".":
            if (i in (0, len(day10data) - 1) or j in (0, len(day10data[0]) - 1) or
                any(day10dataPart2[i+k][j+l][0] == "O" or (-k, -l) in day10dataPart2[i+k][j+l][1] for k, l in nbs)):
                day10dataPart2[i][j][0], c = "O", True
        elif day10dataPart2[i][j][0] in '-|J7LF':
            nn = set((k, l) for k, l in nbs if (k, l) not in tileDirs[day10dataPart2[i][j][0]] and 
                     (i+k in (-1, len(day10data)) or j+l in (-1, len(day10data[0])) or
                      day10dataPart2[i+k][j+l][0] == "O" or (-k, -l) in day10dataPart2[i+k][j+l][1] or
       any((m, n) in day10dataPart2[i][j][1] for m, n in nbs if (m, n) not in tileDirs[day10dataPart2[i][j][0]] and not (m == -k and n == -l)) or
       any((k, l) in day10dataPart2[i+m][j+n][1] for m, n in nbs if abs(k)==abs(n) and abs(l) == abs(m))))
            if nn != day10dataPart2[i][j][1]:
                nn, anyChanges = day10dataPart2[i][j][1].update(nn), True
    if not anyChanges:
        break

In [47]:
sum(1 for i in range(len(day10data)) for j in range(len(day10data[0])) if day10dataPart2[i][j][0] == '.')

423

## Day 11

In [48]:
day11data = getData(11).splitlines()
emptyRows = [yi for yi, y in enumerate(day11data) if "#" not in y]
emptyColumns = [xi for xi in range(len(day11data[0])) if not any(y[xi] == "#" for y in day11data)]
day11data2 = [(yi + sum(1 for el in emptyRows if el < yi), xi + sum(1 for el in emptyColumns if el < xi)) 
             for yi, y in enumerate(day11data) for xi, x in enumerate(y) if x == "#"]

In [49]:
sum(abs(y2-y1)+abs(x2-x1) for (y1, x1), (y2, x2) in itertools.combinations(day11data2, 2))

9769724

In [50]:
day11data2 = [(yi + sum(999999 for el in emptyRows if el < yi), xi + sum(999999 for el in emptyColumns if el < xi)) 
             for yi, y in enumerate(day11data) for xi, x in enumerate(y) if x == "#"]

In [51]:
sum(abs(y2-y1)+abs(x2-x1) for (y1, x1), (y2, x2) in itertools.combinations(day11data2, 2))

603020563700

## Day 12

In [52]:
day12data = [(x.split(" ")[0], tuple(int(y) for y in x.split(" ")[1].split(","))) for x in getData(12).splitlines()]

In [53]:
@functools.lru_cache(maxsize=1000)
def day12recurse(s, ls):
    solution = 0
    matches = re.finditer('(?=([?#]{%d})(?!#))' %ls[0], s) #overlapping, 0-width matches
    for m in matches:
        if not '#' in s[:m.start(1)]: # can't skip any "#". No idea how to do that inside the regexp though
            remaining = s[m.start(1) + ls[0] + 1:].strip(".")
            if len(ls) == 1 and "#" not in remaining:
                    solution += 1
            elif len(ls) > 1 and sum(1 for x in remaining if x in "?#") >= sum(ls[1:]):# still possible?
                    solution += day12recurse(remaining, ls[1:])
    return solution

In [54]:
sum(day12recurse(s,ls) for s, ls in day12data)

7025

In [55]:
sum(day12recurse("?".join(s for _ in range(5)), ls*5) for s, ls in day12data)

11461095383315

## Day 13

In [56]:
day13data = [x.splitlines() for x in getData(13).split("\n\n") if x]

In [57]:
sum(next(itertools.chain((i for i in range(1,len(x[0])) if all(y[i-j] == y[i+j-1] for y in x for j in range(1,1+min(i, len(x[0])-i)))),
                         (100*i for i in range(1,len(x)) if all(a == b for j in range(1, 1+min(i, len(x)-i)) for a, b in zip(x[i-j], x[i+j-1])))))
    for x in day13data)

30802

In [58]:
sum(next(itertools.chain((i for i in range(1,len(x[0])) if sum(y[i-j] != y[i+j-1] for y in x for j in range(1,1+min(i, len(x[0])-i))) == 1),
                         (100*i for i in range(1,len(x)) if sum(a != b for j in range(1, 1+min(i, len(x)-i)) for a, b in zip(x[i-j], x[i+j-1])) == 1)))
    for x in day13data)

37876

## Day 14

In [59]:
day14data = getData(14).splitlines()

In [60]:
for i in range(1, len(day14data)):
    for j in range(len(day14data[0])):
        if day14data[i][j] == "O":
            toMove = next(k for k in range(i+1) if i-k-1 == -1 or day14data[i-k-1][j] in "#O")
            if toMove > 0:
                day14data[i-toMove] = day14data[i-toMove][:j] + "O" + day14data[i-toMove][j+1:]
                day14data[i] = day14data[i][:j] + "." + day14data[i][j+1:]

In [61]:
sum((len(day14data) - i)*sum(1 for x in day14data[i] if x == "O") for i in range(len(day14data)))

108857

In [62]:
def day14rotate(d):
    il, jl = len(d), len(d[0])
    for i, j in ((i, j) for i in range(1, il) for j in range(jl) if d[i][j] == "O"): # North
        if d[i][j] == "O":
            toMove = next(k for k in range(i+1) if i-k == 0 or d[i-k-1][j] in "#O")
            if toMove > 0:
                d[i-toMove] = d[i-toMove][:j] + "O" + d[i-toMove][j+1:]
                d[i] = d[i][:j] + "." + d[i][j+1:]
    for i, j in ((i, j) for i in range(il) for j in range(1, jl) if d[i][j] == "O"): # West
        if d[i][j] == "O":
            toMove = next(k for k in range(j+1) if j-k == 0 or d[i][j-k-1] in "#O")
            if toMove > 0:
                d[i] = d[i][:j-toMove] + "O" + d[i][j-toMove+1:]
                d[i] = d[i][:j] + "." + d[i][j+1:]
    for i, j in ((i, j) for i in range(il-1)[::-1] for j in range(jl) if d[i][j] == "O"): # South
        if d[i][j] == "O":
            toMove = next(k for k in range(il-i+1) if i+k+1 == il or d[i+k+1][j] in "#O")
            if toMove > 0:
                d[i+toMove] = d[i+toMove][:j] + "O" + d[i+toMove][j+1:]
                d[i] = d[i][:j] + "." + d[i][j+1:]
    for i, j in ((i, j) for i in range(il) for j in range(jl-1)[::-1] if d[i][j] == "O"): # East
        if d[i][j] == "O":
            toMove = next(k for k in range(jl-j+1) if j+k+1 == jl or d[i][j+k+1] in "#O")
            if toMove > 0:
                d[i] = d[i][:j+toMove] + "O" + d[i][j+toMove+1:]
                d[i] = d[i][:j] + "." + d[i][j+1:]

In [63]:
day14data = getData(14).splitlines()
loads, i = [], 0
while True:
    day14rotate(day14data)
    loads.append(sum((len(day14data) - i)*sum(1 for x in day14data[i] if x == "O") for i in range(len(day14data))))
    if i > 10:
        try:
            period = next(k for k in range(5, i//2) if all(loads[-j] == loads[-j-k] for j in range(1, k+1)))
            break
        except StopIteration:
            pass
    i += 1

In [64]:
print("Period %d starting after %d, so loads[1000000000] = loads[%d] = %d" %(period, (i-period), z:= (1000000000-i-1) % period + i-period, loads[z]))

Period 14 starting after 111, so loads[1000000000] = loads[117] = 95273


## Day 15

In [65]:
day15data = getData(15).strip().split(",")

In [66]:
sum(functools.reduce(lambda v, c: (v+ord(c))*17 % 256, x, 0) for x in day15data)

513214

In [67]:
boxes = collections.defaultdict(dict) # dicts are insertion-ordered by default
for step in day15data:
    label, fl = step.split("=")[0].split("-")[0], step.split("=")[-1].split("-")[-1]
    box = functools.reduce(lambda v, c: (v+ord(c))*17 % 256, label, 0)
    if step[len(label):len(label)+1] == '-':
        if label in boxes[box]:
            del boxes[box][label]
    else:
        boxes[box][label] = int(fl)
sum((boxNum+1)*(slotNum+1)*fl for boxNum, box in boxes.items() for slotNum, fl in enumerate(box.values()))

258826

## Day 16

In [68]:
day16data = getData(16).splitlines()

In [69]:
day16dirs = {'\\': {(0, 1) : [(1, 0)], (0, -1): [(-1, 0)], (1, 0): [(0, 1)], (-1, 0): [(0, -1)]},
             '/': {(0, 1) : [(-1, 0)], (0, -1): [(1, 0)], (1, 0): [(0, -1)], (-1, 0): [(0, 1)]},
             '|': {(0, 1) : [(1, 0), (-1, 0)], (0, -1): [(1, 0), (-1, 0)], (1, 0): [(1, 0)], (-1, 0): [(-1, 0)]},
             '-': {(0, 1) : [(0, 1)], (0, -1): [(0, -1)], (1, 0): [(0, 1), (0, -1)], (-1, 0): [(0, 1), (0, -1)]}}

In [70]:
def day16fun(startY, startX, dY, dX):
    visited, queue = set(), [(startY, startX, dY, dX)]
    while queue:
        y, x, diry, dirx = queue.pop(0)
        while -1 < y < len(day16data) and -1 < x < len(day16data[0]) and (y, x, diry, dirx) not in visited:
            visited.add((y, x, diry, dirx))
            if day16data[y][x] != '.':
                for diry, dirx in day16dirs[day16data[y][x]][(diry, dirx)]:
                    queue.append((y+diry, x+dirx, diry, dirx))
                break
            y, x = y+diry, x+dirx
    return len(set((y, x) for y, x, diry, dirx in visited))
day16fun(0, 0, 0, 1)

7939

In [71]:
maxScore = 0 # takes 0.5s, not too bad
for yi in range(len(day16data)):
    for xi, dx in [(0, 1), (len(day16data[0]), -1)]:
        maxScore = max(maxScore, day16fun(yi, xi, 0, dx))
for xi in range(len(day16data[0])):
    for yi, dy in [(0, 1), (len(day16data), -1)]:
        maxScore = max(maxScore, day16fun(yi, xi, dy, 0))
maxScore

8318

## Day 17

In [72]:
day17data = [[int(x) for x in y] for y in getData(17).splitlines()]

In [73]:
def day17(minStep, maxStep): #not particularly fast, but it will do.
    dirs, yR, xR, changed = [(0, 1), (-1, 0), (0, -1), (1, 0)], len(day17data), len(day17data[0]), True
    costs = [[dict((i, int(1e10)) for i in range(4)) for x in range(xR)] for y in range(yR)]
    costs[-1][-1].update(dict((i, 0) for i in range(4)))
    while changed:
        changed = False
        for y, x in itertools.product(range(yR)[::-1], range(xR)[::-1]):
            for dirI in range(4):
                nhl = [sum(day17data[y+j*dirs[dirI][0]][x + j*dirs[dirI][1]] for j in range(1, i+1)) + min(costs[ny][nx][(dirI+k) % 4] for k in [-1, 1])
                       for i in range(minStep, maxStep) if -1 < (ny:=y + i*dirs[dirI][0]) < yR and -1 < (nx:= x + i*dirs[dirI][1]) < xR]
                if nhl and min(nhl) < costs[y][x][dirI]:
                    changed = True
                    costs[y][x][dirI] = min(nhl)
    return min(costs[0][0].values())
day17(1, 4)

724

In [74]:
day17(4, 11)

877

## Day 18
I first used the same iterative approach as for day 10, but that blew up for part 2. So I looked up Polygons on wikipedia, and discovered Shoelace formula. Shoelace formula only includes the edge on the left and top, so (by symmetry) we need to add half of the edges +1 back in. This took way longer than it should have.

In [75]:
day18data = [(x[:1], int(x.split(" ")[1])) for x in getData(18).splitlines()]

In [76]:
corners, dirs = [(0, 0)], {"R": (0, 1), "U": (-1, 0), "L": (0, -1), "D": (1, 0)}
for d, l in day18data:
    corners.append((corners[-1][0]+l*dirs[d][0], corners[-1][1]+l*dirs[d][1]))
abs(sum(x1*y2-y1*x2 for (y1, x1), (y2, x2) in zip(corners, corners[1:])))//2 + sum(x[1] for x in day18data)//2 + 1

74074

In [77]:
day18data = [(int(x[-2:-1]), int(x[-7:-2], base=16)) for x in getData(18).splitlines()]

In [78]:
corners, dirs = [(0, 0)], [(0, 1), (1, 0), (0, -1), (-1, 0)] # (y, x), RDLU
for d, l in day18data:
    corners.append((corners[-1][0]+l*dirs[d][0], corners[-1][1]+l*dirs[d][1]))
abs(sum(x1*y2-y1*x2 for (y1, x1), (y2, x2) in zip(corners, corners[1:])))//2 + sum(x[1] for x in day18data)//2 + 1

112074045986829

## Day 19

In [79]:
rules, parts = getData(19).strip().split("\n\n")
for s in "xmas":
    parts = parts.replace(s, '"%s"' %s)
parts = json.loads("[" + parts.replace("\n", ", ").replace("=", ":") + "]")
exec("rules = {" + ", ".join(repr(r.split("{")[0]) + ": lambda x, s, m, a: " + " else ".join([repr(x.split(":")[1]) + " if " + x.split(":")[0]
     if ":" in x else repr(x) for x in r.split("{")[1][:-1].split(",")]) for r in rules.splitlines()) + "}")
rules['A'], rules['R'] = lambda x, s, m, a: x+s+m+a, lambda **args: 0

In [80]:
sol = 0
for p in parts:
    res = rules['in'](**p)
    while isinstance(res, str):
        res = rules[res](**p)
    sol += res
sol

418498

In [81]:
rules = dict((r.split("{")[0], r.split("{")[1].split("}")[0].split(",")) for r in getData(19).strip().split("\n\n")[0].splitlines())

In [82]:
def processRule(rule="in", **args):
    toReturn = []
    for r in rules[rule]:
        if r == "R":
            return toReturn
        elif r == "A":
            return toReturn + [args]
        elif ":" not in r:
            return toReturn + processRule(r, **args)
        else:
            v, ineq, n, newRule, newargs = r[0], r[1], int(r[2:].split(":")[0]), r.split(":")[1], args.copy()
            newargs[v] = range(max(args[v].start, n+1), args[v].stop) if ineq == '>' else range(args[v].start, min(n, args[v].stop))
            if len(args[v]) > 0:
                if newRule == "A":
                    toReturn.append(newargs)
                elif newRule != "R":
                    toReturn.extend(processRule(newRule, **newargs))
            args[v] = range(max(args[v].start, n), args[v].stop) if ineq == '<' else range(args[v].start, min(n+1, args[v].stop))
            if len(args[v]) == 0:
                return toReturn

In [83]:
res = processRule("in", x=range(1, 4001), s=range(1, 4001), m=range(1, 4001), a=range(1, 4001))
sum(math.prod((len(y) for y in x.values())) for x in res)

123331556462603

## Day 20

In [84]:
day20data = dict((x.split(" -")[0].strip("&%"), (x[0] if x[0] in "&%" else "", x.split("> ")[1].split(", "))) for x in getData(20).splitlines())

In [85]:
def processDay20Part1(state):
    queue, pulses = [('broadcaster', False, 'button')], [0, 0]
    while queue:
        current, sig, source = queue.pop(0)
        pulses[1 if sig else 0] += 1
        if current not in day20data:
            state[current] = sig
            continue
        t, dests = day20data[current]
        if t == "":
            queue.extend(((dest, sig, current) for dest in dests))
        elif t == "%":
            if not sig:
                state[current] = not state[current]
                queue.extend(((dest, state[current], current) for dest in dests))
        elif t == "&":
            state[current][source] = sig
            if all(state[current].values()):
                queue.extend(((dest, False, current) for dest in dests))
            else:
                queue.extend(((dest, True, current) for dest in dests))
    return pulses

In [86]:
# False = Low, True = High
state, pulses = dict((m, False if t == "%" else {} if t == "&" else None) for m, (t, c) in day20data.items() if t), [0, 0]
inverters = [m for m, (t, c) in day20data.items() if t == '&']
[state[x].update({m: False}) for m, (t, c) in day20data.items() for x in c if x in inverters]
for i in range(1000):
    p = processDay20Part1(state)
    pulses = [pulses[i] + p[i] for i in range(2)]
math.prod(pulses)

711650489

In [87]:
[(m, (t,c)) for m, (t, c) in day20data.items() if 'rx' in c], [(m, (t,c)) for m, (t, c) in day20data.items() if 'dt' in c]

([('dt', ('&', ['rx']))],
 [('ks', ('&', ['dt'])),
  ('pm', ('&', ['dt'])),
  ('dl', ('&', ['dt'])),
  ('vk', ('&', ['dt']))])

That's a lot of ANDs. Lets find the period of all of these, and we should be able to find the the period of 'rx'

In [88]:
state, i, limit = dict((m, False if t == "%" else {} if t == "&" else None) for m, (t, c) in day20data.items() if t), 0, 2
[state[x].update({m: False}) for m, (t, c) in day20data.items() for x in c if x in inverters]
highs = dict((m, dict((m2, []) for m2 in state[m])) for m, (t, c) in day20data.items() if 'rx' in c)
while any(len(y) <limit for x in highs.values() for y in x.values()):
    i+=1
    queue = [('broadcaster', False, 'button')]
    while queue:
        current, sig, source = queue.pop(0)
        if current in highs and sig and len(highs[current][source]) < limit:
            highs[current][source].append(i)
        if current not in day20data:
            state[current] = sig
            continue
        t, dests = day20data[current]
        if t == "":
            queue.extend(((dest, sig, current) for dest in dests))
        elif t == "%":
            if not sig:
                state[current] = not state[current]
                queue.extend(((dest, state[current], current) for dest in dests))
        elif t == "&":
            state[current][source] = sig
            if all(state[current].values()):
                queue.extend(((dest, False, current) for dest in dests))
            else:
                queue.extend(((dest, True, current) for dest in dests))

In [89]:
offsets = [(y[0],[b-a for a,b in zip(y, y[1:])]) for x in highs.values() for y in x.values()]

In [90]:
math.lcm(*[x[1][0] for x in offsets])

219388737656593

## Day 21

In [91]:
day21data = getData(21).splitlines()
for i in range(64):
    newdata = [y.replace("S", ".").replace("O", ".") for y in day21data]
    for y, x in itertools.product(range(len(day21data)), range(len(day21data[0]))):
        if day21data[y][x] in "SO":
            for yi, xi in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                if newdata[y+yi][x+xi] == ".":
                    newdata[y+yi] = newdata[y+yi][:x+xi] + "O" + newdata[y+yi][x+xi+1:]
    day21data = newdata

In [92]:
sum(1 for y in day21data for x in y if x == "O")

3770

In [93]:
day21data, l, c = getData(21).splitlines(), len(day21data), 2 ## slow...
p2data = [c*y.replace("S", ".") + (y if yi==c else y.replace("S", ".")) + c*y.replace("S", ".") for yi in range(c*2+1) for y in day21data]
countsInShape = [0]
while True:
    newdata, toBreak = [y.replace("S", ".").replace("O", ".") for y in p2data], False
    for y, x in itertools.product(range(len(p2data)), range(len(p2data[0]))):
        if p2data[y][x] in "SO":
            if y in (1, len(p2data)-2) or x in (1, len(p2data)-2):
                toBreak = True
            for yi, xi in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                if newdata[y+yi][x+xi] == ".":
                    newdata[y+yi] = newdata[y+yi][:x+xi] + "O" + newdata[y+yi][x+xi+1:]
    p2data = newdata
    countsInShape.append(sum(1 for y in p2data for x in y if x == "O"))
    if toBreak:
        break

### Off-by-one musings (probably)

Full fields: At steps `i > 128`, there are `f = (i-65)//131` further explored in each cardinal direction. They are arranged in a checkerboard in even/odd tiles. This is a geometric sequence, with solution `((f-1)**2)*[7651, 7699][(i-1) % 2]` and `(f**2)*[7651, 7699][i % 2]`.

New fields are started after 131 steps. On the main lines a new field is started 66 steps earlier, i.e. at 65 vs 131.
on the main lines, a field is started every `(i-64)//131`, and gets finished 194 steps later
so at step i>196, there are 4 fields at stage `s = (i-64) % 131`, and 4 fields at stage `((i-64) % 131 + 131) % 194`
on the diagonals a field is started every `f = i//131` steps, but is finished after 259 steps. so there are up to 2 around in various stages.
there are f fields at stage `i % 131`, and f+1 fields at stage `((i % 131) + 131) % 259` in each quarter.

I couldn't get this too work. The numbers are too large and the example is not helpful to debug these mistakes. Looking at solutions it appears that the solution is even easier since the structure of the grid is actually super symmetrical and hence there is a simple quadratric for these special `i`.

In [94]:
i = 26501365
f = (i - l//2) // l
y0, y1, y2 = [countsInShape[l//2 + i*l] for i in range(3)]
((y2-(2*y1)+y0)//2)*f**2 + (y1-y0-(y2-(2*y1)+y0)//2)*f + y0

628206330073385

## Day 22

In [95]:
class Block:
    def __init__(self, i, coordStr):
        self.i = i
        [setattr(self, n, v) for n, v in zip(["sx", "sy", "sz", "ex", "ey", "ez"], (int(x) for y in coordStr.split("~")  for x in y.split(",")))]
    def __lt__(self, other): 
        return min(self.sz, self.ez) < min(other.sz, other.ez)
    def itercubes(self):
        return itertools.product(*[range(s, e + (e-s)//abs(e-s), (e-s)//abs(e-s)) if e-s else range(s, s+1)
                                  for s, e in [(self.sx, self.ex), (self.sy, self.ey), (self.sz, self.ez)]])
    def __repr__(self):
        return "<Block(%d, \"%d,%d,%d~%d,%d,%d\")>" %(self.i, self.sx, self.sy, self.sz, self.ex, self.ey, self.ez)
    def intersects(self, other, down):
        return self.sz-down <= other.ez and self.ez-down >= other.sz and any((x, y, z-down) in other.itercubes() for (x, y, z) in self.itercubes())
    def moveDown(self, down):
        self.sz, self.ez = self.sz - down, self.ez - down

In [96]:
day22data = sorted([Block(i, l) for i, l in enumerate(getData(22).splitlines())])

In [97]:
stillMoving = True
while stillMoving:
    stillMoving = False
    for j, block in enumerate(day22data):
        steps = next(i for i in range(min(block.sz, block.ez)) if block.sz-i == 1 or any(block.intersects(b, i+1) for b in day22data[:j]))
        if steps > 0:
            stillMoving = True
            block.moveDown(steps)
    day22data = sorted(day22data)

In [98]:
supportedBy = dict((block.i, [b.i for b in day22data[:j] if block.intersects(b, 1)]) for j, block in enumerate(day22data))
supports = dict((block.i, [b.i for b in day22data[j+1:] if block.intersects(b, -1)]) for j, block in enumerate(day22data))
sum(1 for lower, uppers in supports.items() if all(len(supportedBy[upper])>1 for upper in uppers))

507

In [99]:
def desintegrate(blocks):
    disintegrated, newSupportedBy = 0, supportedBy
    while blocks:
        newSupportedBy = dict((k, [y for y in v if y not in blocks]) for k, v in newSupportedBy.items() if v)
        blocks = [k for k, v in newSupportedBy.items() if not v]
        disintegrated += len(blocks)
    return disintegrated

In [100]:
sum(desintegrate([k]) for k, v in supports.items() if v)

51733

## Day 23

In [101]:
day23data = getData(23).splitlines()
yl, xl = len(day23data), len(day23data[0])

In [102]:
queue, bestVal = collections.deque([((0, next(i for i, x in enumerate(day23data[0]) if x == '.')), set())]), 0
while queue:
    (y, x), visited = queue.popleft()
    if y == yl-1:
        bestVal = max(bestVal, len(visited))
    else:
        visited.add((y, x))
        nextSteps = [(y+yi, x+xi) for yi, xi, wf in [(0, 1, ">"), (1, 0, "v"), (-1, 0, "^"), (0, -1, "<")] if day23data[y+yi][x+xi] in (".", wf)
                    and (y+yi, x+xi) not in visited]
        if nextSteps:
            queue.append((nextSteps[0], visited))
        if len(nextSteps) > 1:
            queue.extend((x, visited.copy()) for x in nextSteps[1:])
bestVal

2394

In [103]:
nodes = collections.defaultdict(list)
edges = collections.defaultdict(lambda : collections.defaultdict(list))
queue = collections.deque([((1, y), [(0, y)]) for y in (i for i, x in enumerate(day23data[0]) if x == '.')])
edgeCount = 0
while queue:
    (y, x), visited = queue.popleft()
    if (y,x) in nodes[visited[0]]:
        continue
    if len(visited) == 1:
        nodes[visited[0]].append((y,x))
    visited.append((y, x))
    if y == yl-1:
        edges[visited[0]][(y, x)].append((edgeCount, len(visited)-1))
        edges[(y, x)][visited[0]].append((edgeCount, len(visited)-1))
        edgeCount += 1
        nodes[(y, x)].append(visited[-2])
    else:
        nextSteps = [(y+yi, x+xi) for yi, xi in [(0, 1), (1, 0), (-1, 0), (0, -1)] if day23data[y+yi][x+xi] in ".<>^v" and (y+yi, x+xi) not in visited]
        if len(nextSteps) == 1:
            queue.appendleft((nextSteps[0], visited))
        else:
            if visited[-2] not in nodes[(y, x)]:
                edges[visited[0]][(y, x)].append((edgeCount, len(visited)-1))
                edges[(y, x)][visited[0]].append((edgeCount, len(visited)-1))
                edgeCount += 1
                nodes[(y, x)].append(visited[-2])
                queue.extendleft((n, [(y, x)]) for n in nextSteps if n not in nodes[(y, x)])

In [104]:
#Slow, but works.
queue, bestVal = collections.deque([((0, next(i for i, x in enumerate(day23data[0]) if x == '.')), set(), 0)]), 0
while queue:
    (y, x), visited, score = queue.popleft()
    if y == yl-1:
        bestVal = max(bestVal, score)
    else:
        visited.add((y, x))
        nextSteps = [(n, c) for n, c in edges[(y,x)].items() if n not in visited]
        if nextSteps:
            queue.appendleft((nextSteps[0][0], visited, score+nextSteps[0][1][0][1]))
        if len(nextSteps) > 1:
            queue.extendleft((x[0], visited.copy(), score+x[1][0][1]) for x in nextSteps[1:])
bestVal

6554

## Day 24

In [105]:
day24data = [(tuple(int(y) for y in x.split("@")[0].split(",")), tuple(int(y) for y in x.split("@")[1].split(","))) for x in getData(24).splitlines()]

In [106]:
s = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3"""
day24data = [(tuple(int(y) for y in x.split("@")[0].split(",")), tuple(int(y) for y in x.split("@")[1].split(","))) for x in s.splitlines()]

In [107]:
def doIntersect(a, va, b, vb, minc=-math.inf, maxc=math.inf):
    x12, y34, y12, x34 = a[0]-(a[0]+va[0]), b[1]-(b[1]+vb[1]), a[1]-(a[1]+va[1]), b[0]-(b[0]+vb[0])
    det = x12*y34-y12*x34
    if det == 0: return False, 0, 0
    px = ((a[0]*(a[1]+va[1])-a[1]*(a[0]+va[0]))*x34-x12*(b[0]*(b[1]+vb[1])-b[1]*(b[0]+vb[0])))/det
    py = ((a[0]*(a[1]+va[1])-a[1]*(a[0]+va[0]))*y34-y12*(b[0]*(b[1]+vb[1])-b[1]*(b[0]+vb[0])))/det
    return minc <= px <= maxc and minc <= py <= maxc and (va[0] == 0 or (px-a[0])/va[0] >= 0) and (va[1] == 0 or (py-a[1])/va[1] >= 0) \
                                                     and (vb[0] == 0 or (px-b[0])/vb[0] >= 0) and (vb[1] == 0 or (py-b[1])/vb[1] >= 0), px, py

In [108]:
sum(doIntersect(*a, *b, 200000000000000, 400000000000000)[0] for a, b in itertools.combinations(day24data, 2))

0

Yay, linear algebra. The solution is pretty obvious (just write down some straight foward equations and solve them), but painful to write out without additional library, hence I am taking inspiration from werner77's solution below.

Not finished yet, I still need to implement the vector projections. They need to be applied to specific dimensions, rather than just selecting dimensions
* https://en.wikipedia.org/wiki/Vector_projection
* https://math.stackexchange.com/questions/458843/projection-of-high-dimensional-vectors-to-lower-dimensional-space
* https://github.com/werner77/AdventOfCode/blob/master/src/main/kotlin/com/behindmedia/adventofcode/year2023/day24/Day24.kt

In [109]:
def bruteForce(dimToIgnore):
    maxVelo = 400
    dims = [i for i in range(3) if i != dimToIgnore]
    for vx, vy in itertools.product(range(-maxVelo, maxVelo),repeat=2):
        intersections = []
        for (a, av), (b, bv) in itertools.combinations(day24data, 2):
            intersec, px, py = doIntersect([a[i] for i in dims], [av[i]+v for i, v in zip(dims, [vx, vy])],
                                           [b[i] for i in dims], [bv[i]+v for i, v in zip(dims, [vx, vy])])
            if intersec:
                intersections.append((px, py))
            if len(intersections) > 4 and all(abs(x-intersections[0][0])<1 and abs(y-intersections[0][1])<1 for x,y in intersections[1:]):
                return intersections[0], (-vx, -vy)
            else:
                break
    return False

In [110]:
bruteForce(0)

False

In [111]:
#bruteForce(1)

In [112]:
#bruteForce(2)